In [186]:
import pandas as pd
import os
from datetime import datetime, timedelta

config_file_path = 'Config.csv'

In [187]:
def set_flags(x,limit_type,r_limit,y_limit):
    flag = 'G'
    if limit_type == 'UCL':
        if x >= r_limit:
            flag = 'R'
        elif x>= y_limit:
            flag = 'Y'
    else:
        if x <= r_limit:
            flag = 'R'
        elif x<= y_limit:
            flag = 'Y'        
    return flag

def load_prod_data(server,product,details,limits,debug=False):
    tech = details['TECH']
    prod = details['PART']
    rev = details['REV']
    
    folders = r'\Actuals\Last_49_Days'
    if tech=='P1273':
        folders = r'\Actuals\Last_49Days'
    
    file_path = os.path.join(r'\\'+server,tech+r'_Data'+folders,prod+'.csv')
    
    # debug = True
    df = pd.read_csv(file_path)
    # if debug: print(df.head())

    if debug:
        col_check = 1
        num_rows = df.shape[0]
        print(f"Column check {col_check} = {num_rows}")
        col_check +=1
        
    df = df[df['PROCESS_REV'] == rev]

    if debug:
        num_rows = df.shape[0]
        print(f"Column check {col_check} = {num_rows}")
        col_check +=1

    df['SORT_DATE'] = pd.to_datetime(df['SORT_DATE'], errors='coerce')
    # Calculate the start and end dates for the last 4 full weeks
    today = datetime.today()
    start_of_this_week = today - timedelta(days=today.weekday() + 1)
    start_of_4th_last_full_week = start_of_this_week - timedelta(weeks=4)

    # Filter the DataFrame for the last 4 full weeks
    df = df[
        (df['SORT_DATE'] >= start_of_4th_last_full_week)
    ]
    if debug:
        num_rows = df.shape[0]
        print(f"Column check {col_check} = {num_rows}")
        col_check +=1

    possible_columns = ['IDV', 'SICC', 'CAPABILITY', 'CDYN']
    columns_of_interest = []
    
    for col in possible_columns:        
        if pd.notna(details[col]) and details[col] != '':
            columns_of_interest.append(col)
            df[col] = df[details[col]]
    
    means = df.groupby('FAB')[columns_of_interest].mean().reset_index()
    means['TECH'] = tech
    means['PRODUCT'] = product
    
    # for col in columns_of_interest:
        # means[col+'_Flag'] = 'Green'
        
    for col in columns_of_interest:
    # print(col)
        limit_type = limits[col+'_TYPE']
        target = details[col+'_TGT']
        r_limit = target*(1+limits[col+'_RED'])
        y_limit = target*(1+limits[col+'_YELLOW'])
        means[col+'_FLAG'] = means[col].apply(set_flags, args=(limit_type, r_limit, y_limit))
    

    
    return means
    

In [188]:
def load_excel_to_dict(file_path,key_col):
    df = pd.read_csv(file_path)
    temp_dict = {}
    for _, row in df.iterrows():
        key = row[key_col]
        temp_dict[key] = row.drop(key_col).to_dict()
    return temp_dict

In [189]:
prod_dict = load_excel_to_dict('Config.csv','PRODUCT')
limit_dict = load_excel_to_dict('Tech_Limits.csv','TECH')

In [190]:
server = 'rasinkul-desk'

temp_dfs = []
for product, details in prod_dict.items():
    print(product)
    limits = limit_dict[details['TECH']]
    product = details['PART']
    
    temp_df = load_prod_data(server=server, product=product, details=details,limits=limits, debug=False)
    
    temp_dfs.append(temp_df)
    
final_df = pd.concat(temp_dfs, ignore_index=True)


ADL PCH-P
RPL816
RPL68
SPRXCC
MTL68


C:\Users\dagarcia\AppData\Local\Temp\ipykernel_14376\2419402945.py:27: DtypeWarning: Columns (442,443,444,445,446,447) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


GNRXCC


C:\Users\dagarcia\AppData\Local\Temp\ipykernel_14376\2419402945.py:27: DtypeWarning: Columns (442,443,444,445,446,447) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [191]:
desired_order = ['TECH','PRODUCT','FAB','IDV','SICC','CAPABILITY','CDYN','IDV_FLAG','SICC_FLAG','CAPABILITY_FLAG','CDYN_FLAG']
    
final_df = final_df[desired_order]
final_df

,TECH,PRODUCT,FAB,IDV,SICC,CAPABILITY,CDYN,IDV_FLAG,SICC_FLAG,CAPABILITY_FLAG,CDYN_FLAG
0,P1273,88ABCVB,F24,9456.014293,0.093913,9456.975492,NaN,G,G,G,NaN
1,P1274,8PQFCVB,F28,16677.111574,0.035028,16606.872139,3.505001,G,G,G,G
2,P1274,8PQFCVB,F32,16695.906671,0.035333,16617.770124,3.487614,G,Y,G,G
3,P1274,8PQGCVJ,F28,16291.937906,0.014773,16247.468240,3.257808,G,G,G,G
4,P1274,8PQGCVJ,F32,16254.363080,0.014518,16221.491805,3.243383,G,G,G,G
5,P1274,8PSXCVE,F32,14811.619478,13.265769,14907.097940,2.771435,G,G,G,G
6,P1276,8PRPCVS,D1D,1.000117,0.100940,1.001717,2.199054,G,G,G,G
7,P1276,8PRPCVS,F34,1.008371,0.118977,0.999208,2.220750,G,Y,G,G
8,P1276,8PG3CVB,D1D,1.095329,1.035021,1.090511,2.085386,G,G,G,G
9,P1276,8PG3CVB,F34,1.105341,1.091844,1.091329,2.088717,G,Y,G,G
